<img src="header.png" align="left"/>

# Exercise Import of excel data and timeseries prediction (10 points)

The goal of this example is to show the work with time series from Excel files and to develop a prediction model for time series. The data set describes the development of passenger numbers of an airline in the distant past. However, the data is typical for data as found in SME's (e.g. sales figures). 


The code for this example was adapted from [1]. Further information is available here [2]. 

- [1] [https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
- [2] [https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa](https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa)


Citation dataset:
```
Box, G. E. P., Jenkins, G. M. and Reinsel, G. C. (1976) Time Series Analysis, Forecasting and Control. Third Edition. Holden-Day. Series G.
```


# Import of modules

In [ ]:
import os
import openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

plt.rcParams["figure.figsize"] = (16,9)

In [ ]:
#
# handle error messages
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)

In [ ]:
#
# Für GPU Support
#
import tensorflow as tf
print ( tf.__version__ ) 

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR )
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Constants

In [ ]:
excelData = 'data/airline_passengers.xlsx'

# Read the data from Excel file

In [ ]:
data = pd.ExcelFile(excelData)
print(data.sheet_names)

In [ ]:
#
# read one table
#
df = data.parse('Tabellenblatt1')
df.info
df.head(10)

In [ ]:
df.tail()

# Cut out data from pandas dataframes

Tutorial: [https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)

<img src="info.png" align="left"/> 

In [ ]:
#
# cut out data
#
# Task: cut out the data from the dataframe using the iloc function described above to have 
# only relevant data left (timestamp and passenger number, no NaNs) (2 points)
#

df_cut = ...


In [ ]:
df_cut.head()

In [ ]:
# rename columns to fixed names
df_cut.columns = ['month','passengers']

In [ ]:
df_cut.head()

In [ ]:
#
# plot number of passangers
#
# Task: plot the passenger data from the pandas dataframe (1 point)
#

...

# Conventional methods

There are many older methods to work with time series:

- https://machinelearningmastery.com/time-series-trends-in-python/
- https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa


The main purpose of these methods is the calculation and use of parameters such as **trend** and **seasonality**. We hope that our models can handle them without us having to explicitly deal with them.

<img src="info.png" align="left"/> 

In [ ]:
#
# prepare data
#
x_data = df_cut['passengers'].values
x_data = x_data.astype('float32')
x_data = np.reshape(x_data,(-1,1))
print(x_data)

In [ ]:
#
# Normalize the dataset
#
# Task: create a minmax scaler from sklearn to scale the data between 0 and 1 and store the trained scaler in
# a python variable called scaler (2 points)
#
scaler = ...
scaler.fit(x_data)

In [ ]:
print(scaler.data_max_)

In [ ]:
#
# actually scale the data
#
x_data = scaler.transform(x_data)
print(x_data)

In [ ]:
#
# split into train and test sets with 80% training data
#
train_size = int(len(x_data) * 0.80)
test_size = len(x_data) - train_size
train, test = x_data[0:train_size,:], x_data[train_size:len(x_data),:]
print(len(train), len(test))

In [ ]:
#
# plot scaled training data for check
#
plt.plot(train)

# Create a training data set with sliding windows

Sliding windows are created from a time series by dragging a window over the entire time series and copying the data in the window. More details here [https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe](https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe).

<img src="info.png" align="left"/> 

In [ ]:
#
# create sliding windows with one label (y)
#
# Task: add comments for the details of this function (1 point)
#
def createSlidingWindowsWithLabel(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[ i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
#
# apply sliding window function with window size (window_length)
#
window_length = 1
trainX, trainY = createSlidingWindowsWithLabel(train, window_length)
testX, testY = createSlidingWindowsWithLabel(test, window_length)

In [ ]:
# 
# reshape input to be [samples, time steps, features]
#
trainX = np.reshape(trainX, (trainX.shape[0], window_length, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], window_length, testX.shape[1]))

# Create an LSTM model

In [ ]:
# some hyperparameters
epochs = 100
batch_size = 4

In [ ]:
#
# create and fit the LSTM network
#
def createLSTMModel():
    model = Sequential()
    model.add(LSTM(6, input_shape=(1, window_length)))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
lstm_model = createLSTMModel()

In [ ]:
lstm_model.summary()

In [ ]:
history = lstm_model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(testX, testY), shuffle=False)

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
#
# Estimate values for train and test data
#
trainPredict = lstm_model.predict(trainX)
testPredict = lstm_model.predict(testX)

In [ ]:
#
# Inverse transformation of estimations (scaler)
#
trainPredicti = scaler.inverse_transform(trainPredict)
testPredicti = scaler.inverse_transform(testPredict)

trainYi = scaler.inverse_transform([trainY])
testYi = scaler.inverse_transform([testY])

In [ ]:
#
# Plot estimations
#
plt.plot(testYi[0,0:])
plt.plot(testPredicti[1:,0])
plt.show()

In [ ]:
#
# calculate root mean squared error
#
# Task: calculate the root mean squared error between the test labels and the prediction (1 point)
# Hint: labels are in testYi[0,0:-1]
#       predictions are in testPredicti[1:,0]

trainScore = ...
print('train loss: %.3f RMSE' % (trainScore))
testScore = ...
print('test loss: %.3f RMSE' % (testScore))

# Plot complete timeline

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[:len(trainPredicti)+0, :] = trainPredicti

# shift test predictions for plotting
testPredictPlot = np.empty_like(x_data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredicti)+(look_back*2):len(x_data)-2, :] = testPredicti

In [ ]:
# plot baseline and predictions
plt.plot(scaler.inverse_transform(x_data), color='grey')
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

# Experiment with different window and model sizes (3 points)

**change request**: The customer wants a better prediction quality. Conduct an experiment with a different window_length. Another option is to increase the capacity of the model. Compare the resulting RMSE values. 
Hopefully, this increases the quality of the prediction.

In [ ]:
#
# apply sliding window function with window size (window_length)
#
window_length = ?
trainX, trainY = createSlidingWindowsWithLabel(train, window_length)
testX, testY = createSlidingWindowsWithLabel(test, window_length)

In [ ]:
# check shape of windows
print(trainX.shape)

In [ ]:
print(trainX)